In [1]:
import random
import os

import numpy as np
from matplotlib import pyplot as plt
import cv2
from tensorpack import dataflow
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import clear_output

from nets import G, D
from parameters import *

%matplotlib inline

Failed to import tensorflow.


In [2]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor

if USE_CUDA:
    print('Using CUDA')
else:
    print('Using CPU')

Using CUDA


In [3]:
if not os.path.exists(DATA_PATH):
    print('Can\'t find DATA_PATH:', DATA_PATH) 
    quit(1)

In [4]:
if DATASET_SIZE == 0:
    print('No dataset found')
    quit(1)

In [5]:
def path_iter():
    filenames = os.listdir(DATA_PATH)
    
    while True:
        random.shuffle(filenames)
        
        for fn in filenames:
            yield os.path.join(DATA_PATH, fn)

In [6]:
def prepare_image(path):
    image = cv2.imread(path)
    
    if image is None:
        return None
    
    if image.shape[0] != 64 or image.shape[1] != 64 or image.shape[2] != 3:
        print('Bad image shape')
        quit(1)
    
    image = np.moveaxis(image, 2, 0)
    image = ((image / 255.) - .5) * 2
        
    return image

In [7]:
image_iter = dataflow.MapData(path_iter(), func=prepare_image)

In [8]:
ngf = 64
nc = 3

In [9]:
class G(nn.Module):
    def __init__(self):
        super(G, self).__init__()
        
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(PRIOR_DIM, ngf * 8, 4, 1, 0, bias=False),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )
        
        if USE_CUDA:
            self.cuda()
            
    def forward(self, x):
        x = torch.reshape(x, (-1, PRIOR_DIM, 1, 1))
        
        x = self.main(x)
        
        return x


In [10]:
ndf = 64

In [11]:
class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
        if USE_CUDA:
            self.cuda()
        
    def forward(self, x):
        x = self.main(x)
        
        return x.view((-1, 1))
    
    def clip(self):
        self.main[0].weight.data.clamp_(min=-CLIP, max=CLIP)
        self.main[2].weight.data.clamp_(min=-CLIP, max=CLIP)
        self.main[4].weight.data.clamp_(min=-CLIP, max=CLIP)
        self.main[6].weight.data.clamp_(min=-CLIP, max=CLIP)
        self.main[8].weight.data.clamp_(min=-CLIP, max=CLIP)

In [12]:
def init_weights(m):
    if type(m) in [nn.Conv2d, nn.Linear, nn.ConvTranspose2d]:
        torch.nn.init.normal_(m.weight.data, mean=0, std=INIT_STD)
        if m.bias is not None:
            torch.nn.init.normal_(m.bias.data, mean=0, std=INIT_STD)
    elif type(m) in [nn.BatchNorm2d, nn.LeakyReLU, nn.ReLU, nn.Sequential, nn.Tanh]:
        return
    else:
        print('Couldn\'t init wieghts of layer with type:', type(m))


In [13]:
generator = G()
discriminator = D()

generator.apply(init_weights)
discriminator.apply(init_weights);

Couldn't init wieghts of layer with type: <class '__main__.G'>
Couldn't init wieghts of layer with type: <class 'torch.nn.modules.activation.Sigmoid'>
Couldn't init wieghts of layer with type: <class '__main__.D'>


In [14]:
G_optim = optim.RMSprop(generator.parameters(), lr = LR)
D_optim = optim.RMSprop(discriminator.parameters(), lr = LR)

In [15]:
def prior():
    #return np.random.multivariate_normal(np.zeros(PRIOR_DIM), np.identity(PRIOR_DIM))
    return np.random.randn(PRIOR_DIM)

In [16]:
def get_data_minibatch():
    i = 0
    ret = []
    for image in image_iter:
        ret.append(image)
        
        i += 1
        if i == MINIBATCH_SIZE:
            break
    
    return np.stack(ret, axis=0)

In [17]:
start_epoch = 0

In [18]:
def save(fn=None, epoch_num=None):
    if fn is None:
        fn = 'checkpoint'

    torch.save({
        'gen': generator.state_dict(),
        'dis': discriminator.state_dict(),
        'g_opt': G_optim.state_dict(),
        'd_opt': D_optim.state_dict(),
        'last_epoch' : epoch_num
    }, fn)

In [19]:
def load(fn):
    if fn is None:
        fn = 'checkpoint'
    state = torch.load(fn)

    generator.load_state_dict(state['gen'])
    discriminator.load_state_dict(state['dis'])
    G_optim.load_state_dict(state['g_opt'])
    D_optim.load_state_dict(state['d_opt'])
    
    if not(state['last_epoch'] is None):
        start_epoch = state['last_epoch'] + 1

In [20]:
PLOT_FREQ = 100
GEN_FREQ = 100

if not os.path.exists('generated'):
    os.mkdir('generated')
    
if os.path.exists('checkpoint'):
    load('checkpoint')
    print('Loaded checkpoint')
else:
    print('Cold start')    

generator.train()
discriminator.train()
    
plot_iter = 0
gen_iter = 0
g_losses = []
d_losses = []
for epoch_num in range(start_epoch, EPOCH_COUNT):
    print('===============================')
    print('Epoch', epoch_num, 'started!')
    print('===============================')
    
    for t in tqdm(range(EPOCH_LEN)):     
        min_d_loss = 1e100
        
        for k in range(DISCRIMINATOR_LEARNING_REPEATS):
            D_optim.zero_grad()
            
            mean_d_loss = 0

            for sample_num in range(BATCH_DIVISOR):
                data = FloatTensor(get_data_minibatch())
                p = FloatTensor(np.stack([prior() for i in range(MINIBATCH_SIZE)], axis=0))

                D_of_x = discriminator(data)
                D_of_G_of_z = discriminator(generator(p))

                loss = D_of_x - D_of_G_of_z
                loss = torch.mean(loss, dim = 0)
                loss = -loss
                loss = loss / BATCH_DIVISOR

                loss.backward()
                
                mean_d_loss += loss.item()
                
            D_optim.step()
            
            discriminator.clip()
            
            mean_d_loss /= BATCH_DIVISOR
            min_d_loss = min(mean_d_loss, min_d_loss)
        
        #=========================================
        
        G_optim.zero_grad()

        mean_g_loss = 0
        
        for sample_num in range(BATCH_DIVISOR):
            p = FloatTensor(np.stack([prior() for i in range(MINIBATCH_SIZE)], axis=0))

            D_of_G_of_z = discriminator(generator(p))

            loss = -D_of_G_of_z
            loss = torch.mean(loss, dim = 0)
            loss = loss / BATCH_DIVISOR

            loss.backward()
        
            mean_g_loss += loss.item()

        G_optim.step()
        
        mean_g_loss /= BATCH_DIVISOR
        
        #=========================================
        
        g_losses.append(mean_g_loss)
        d_losses.append(mean_d_loss)
        
        if gen_iter % GEN_FREQ:
            generator.eval()        
            image = (generator(FloatTensor(np.expand_dims(prior(), 0))).detach().cpu().numpy()[0] + 1) / 2
            image = (np.moveaxis(image, 0, 2) * 255).astype(np.uint8)
            cv2.imwrite(os.path.join('generated', str(t % 16) + '.jpg'), image)
            generator.train()
        
        if plot_iter % PLOT_FREQ == 0:
            fig = plt.figure(figsize=(8, 6), dpi=80)
            ax = fig.add_subplot()
            ax.plot(g_losses, label='G losses')
            ax.plot(d_losses, label='D losses')
            ax.legend()
            fig.savefig('D&G.png')
            fig.clear()
            plt.close(fig)
            
        gen_iter += 1
        plot_iter += 1
    
    print('Saving...')
    save('checkpoint', epoch_num)

Cold start
Epoch 0 started!


100%|██████████████████████████████████████████████████████████████████████████| 19373/19373 [1:56:25<00:00,  2.77it/s]


Saving...
Epoch 1 started!


100%|██████████████████████████████████████████████████████████████████████████| 19373/19373 [1:58:11<00:00,  2.73it/s]


Saving...
Epoch 2 started!


100%|██████████████████████████████████████████████████████████████████████████| 19373/19373 [2:08:14<00:00,  2.52it/s]


Saving...
